In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup
import os
import ast
import datetime

# Code to dynamically extract the data. This code has to run 6 times for each page in metro cuadrado

 ### Date of the last Capture: 09/22/2024   , City: Tunja

In [60]:
import platform
print(platform.architecture())


('64bit', 'WindowsPE')


In [63]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Initialize the ChromeDriver properly
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [64]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

from selenium import webdriver
from selenium.webdriver.common.by import By

# Define the URL
url = "https://www.metrocuadrado.com/venta/tunja"

# Load the web page
driver.get(url)

# Set maximum time to wait for elements to load in seconds (implicitly_wait)
driver.implicitly_wait(10)

# Find the container that holds the list of houses
contents = driver.find_element(By.ID, "__next")

parts = url.split("/")
city_name = parts[-1]
city_name

'tunja'

### I have to press next in the website for the next houses.

In [92]:
# Find all the <li> elements within the container
metadata_2 = contents.find_element(By.CSS_SELECTOR,'div > div > div.Layout__LayoutStyled-sc-9y7jis-0.ibZBWk.page-container > div.Container-u38a83-0.jDuhNh.inner-container.container > div:nth-child(2) > div.Col-sc-14ninbu-0.lfGZKA.col-md-8.col-lg-9 > ul.Ul-sctud2-0.jyGHXP.realestate-results-list.browse-results-list')

links = metadata_2.find_elements(By.TAG_NAME, "a")

list_links = []
for element in links:
    link = element.get_attribute('href')
    list_links.append(link)

list_links
list_links_8 =list_links ### Change the number at the end of the name when you want to change the page

In [93]:
#### Check each list
list_links[0:5]

['https://www.metrocuadrado.com/inmueble/venta-casa-tunja-josa%C2%BF-antonio-gala%C2%81n-3-habitaciones-2-banos-2-garajes/10159-M4937798',
 'https://www.metrocuadrado.com/inmueble/venta-casa-tunja-josa%C2%BF-antonio-gala%C2%81n-3-habitaciones-2-banos-2-garajes/10159-M4937798',
 None,
 'https://www.metrocuadrado.com/inmueble/venta-lote-o-casalote-tunja-centro-historico-uptc-medicina/MC4869813',
 'https://www.metrocuadrado.com/inmueble/venta-lote-o-casalote-tunja-centro-historico-uptc-medicina/MC4869813']

In [94]:
### Combined Observations ###

combined_list = []
n = 8  # You can change this to the number of your lists

for i in range(1, n + 1):
    list_name = globals().get(f'list_links_{i}')
    
    if list_name is not None:
        combined_list.extend(list_name)

### Drop Repeated Observations ###

unique_list = list(set(combined_list))


In [95]:
len(unique_list)

371

# Following part is to capture the data from each link with beautifull soup

In [96]:
from bs4 import BeautifulSoup
import requests
import re

In [97]:
# List of URLs
url_list = [url for url in unique_list if url is not None]  

# Initialize lists to store data
name_list = []
id_list = []
barrio_list = []
precio_list = []
edad_list = []
are_construida_list = []
area_total_list = []
parking_list = []
descripcion_list = []
estrato_list = []
hab_list = []
bath_list = []
other_variable_list = []

# Loop through the URLs
for index,url in enumerate(url_list):
    print(f"Processing URL at position {index}: {url}")
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')

        ### Location of data
        name_call = soup.select('.jdfXCo')
        first_call = soup.select(".col-lg-3 .card-text")
        second_call = soup.select('.ctJpwh .py-2')
        third_call = soup.select('.w-90 .card-text')

        name = name_call[0].text
        id = first_call[0].text
        barrio = first_call[1].text
        precio = first_call[2].text
        edad = first_call[3].text
        are_construida = first_call[4].text
        
        if len(first_call) > 5:
                area_total = first_call[5].text
        else:
                area_total = None

        if len(first_call) > 6:
                parking = first_call[6].text
        else:
                parking = None
        
        descripcion = soup.select(".d-md-block")[0].text

        second_call_list = [item.text for item in second_call]

        estrato = None
        hab = None
        bath = None

        for item in second_call_list:
            if "Estrato" in item:
                estrato = item.split("E")[0]
            if "Baños" in item:
                bath = item.split("B")[0]
            if "Habitaciones" in item:
                hab = item.split("H")[0]

        third_call_list = [item.text for item in third_call]
        other_variable = str(third_call_list)

        # Append the extracted values to their respective lists
        name_list.append(name)
        id_list.append(id)
        barrio_list.append(barrio)
        precio_list.append(precio)
        edad_list.append(edad)
        are_construida_list.append(are_construida)
        area_total_list.append(area_total)
        parking_list.append(parking)
        descripcion_list.append(descripcion)
        estrato_list.append(estrato)
        hab_list.append(hab)
        bath_list.append(bath)
        other_variable_list.append(other_variable)

    except IndexError:
         print(f"Index error for URL at position {index}: {url}. Skipping to the next URL.")
         continue

# Create a DataFrame from the collected data
data = {
    'Name': name_list,
    'ID': id_list,
    'Barrio': barrio_list,
    'Precio': precio_list,
    'Edad': edad_list,
    'Area Construida': are_construida_list,
    'Area Total': area_total_list,
    'Parking/admin_value': parking_list,
    'Descripcion': descripcion_list,
    'Estrato': estrato_list,
    'Habitaciones': hab_list,
    'Baños': bath_list,
    'Other Variable': other_variable_list
}

df = pd.DataFrame(data)


Processing URL at position 0: https://www.metrocuadrado.com/inmueble/venta-local-comercial-tunja-venta-de-local-ubicado-en-el-sector-av-universitaria-en-tunja-boyaca/11295-M3409992
Processing URL at position 1: https://www.metrocuadrado.com/inmueble/venta-lote-o-casalote-tunja-vereda-el-porvenir/11295-M3860104
Processing URL at position 2: https://www.metrocuadrado.com/inmueble/venta-apartamento-tunja-san-ignacio-2-habitaciones-1-banos-1-garajes/16742-M5249136
Processing URL at position 3: https://www.metrocuadrado.com/inmueble/venta-apartamento-tunja-mezopotamia-3-habitaciones-3-banos-1-garajes/10159-M5082691
Processing URL at position 4: https://www.metrocuadrado.com/inmueble/venta-apartamento-tunja-3-habitaciones-2-banos-1-garajes/3072-M5257843
Processing URL at position 5: https://www.metrocuadrado.com/inmueble/venta-casa-tunja-centro-hista%C2%BFrico-5-habitaciones-5-banos/10159-M4838438
Processing URL at position 6: https://www.metrocuadrado.com/inmueble/venta-casa-tunja-puente-ha

In [98]:
### Create the other variables based in the categories extracted ##


# Function to convert the string representation of lists into actual lists
def parse_list(string_list):
    try:
        return ast.literal_eval(string_list)
    except (ValueError, SyntaxError):
        return []

# Apply the parse_list function to convert strings to lists
df['list'] = df['Other Variable'].apply(parse_list)

# Flatten the lists and duplicate the values
all_values = list(set([value for sublist in df['list'] for value in sublist]))

# Create dummy variables for each possible value
for value in all_values:
    df[value] = df['Other Variable'].str.contains(value).astype(int)

### Keep only the categories that are repeated more than 4 times 

summary_row = df.sum().reset_index()
summary_row[0] = pd.to_numeric(summary_row[0], errors='coerce')

filtered = summary_row[summary_row[0]<5].sort_values(by = 0,ascending=True)
index_list = filtered['index'].tolist()

data_set_filtered = df.drop(columns = index_list)
data_set_filtered['date'] = datetime.date.today()
data_set_filtered['city'] = city_name

# Get the current date and time
current_date = datetime.date.today()

# Construct the filename
filename = f"{city_name}_{current_date}.csv"

# Save the DataFrame to the CSV file
os.chdir('C:/Users/genin/Box Sync/MY DOCUMENTS/Proyecto Casas e Inmuebles/data capturas')
data_set_filtered.to_csv(filename, index=False)

C:\Users\genin\AppData\Local\Temp\ipykernel_21692\2586265256.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[value] = df['Other Variable'].str.contains(value).astype(int)
C:\Users\genin\AppData\Local\Temp\ipykernel_21692\2586265256.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[value] = df['Other Variable'].str.contains(value).astype(int)
C:\Users\genin\AppData\Local\Temp\ipykernel_21692\2586265256.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` ma